In [20]:
import re
import csv
import numpy as np
import pandas as pd
import nltk
from sklearn.preprocessing import LabelEncoder
df= pd.read_csv('amazon_alexa.tsv',delimiter='\t',quoting=3)
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"""Sometimes while playing a game, you can answe...",1
3,5,31-Jul-18,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [21]:
df.describe()

,rating,feedback
count,3150.000000,3150.000000
mean,4.463175,0.918413
std,1.068506,0.273778
min,1.000000,0.000000
25%,4.000000,1.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,1.000000


In [22]:
print(df.dtypes)
df.variation.unique()

rating               int64
date                object
variation           object
verified_reviews    object
feedback             int64
dtype: object


array(['Charcoal Fabric ', 'Walnut Finish ', 'Heather Gray Fabric ',
       'Sandstone Fabric ', 'Oak Finish ', 'Black', 'White',
       'Black  Spot', 'White  Spot', 'Black  Show', 'White  Show',
       'Black  Plus', 'White  Plus', 'Configuration: Fire TV Stick',
       'Black  Dot', 'White  Dot'], dtype=object)

In [29]:
lbl = LabelEncoder()
dfNew = df.drop(columns=['date'])
display(df.head())
dfNew['variation'] = lbl.fit_transform(dfNew['variation'])
dfNew = pd.get_dummies(data=dfNew,columns=['variation'])
dfNew.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"""Sometimes while playing a game, you can answe...",1
3,5,31-Jul-18,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1


,rating,verified_reviews,feedback,variation_0,variation_1,variation_2,variation_3,variation_4,variation_5,variation_6,variation_7,variation_8,variation_9,variation_10,variation_11,variation_12,variation_13,variation_14,variation_15
0,5,Love my Echo!,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,5,Loved it!,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,4,"""Sometimes while playing a game, you can answe...",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,5,"""I have had a lot of fun with this thing. My 4...",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Music,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [100]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as wt
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
#nltk.download('wordnet')
lm = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
stop_words = set(stopwords.words('english'))
wrdList=[]
for each in dfNew.verified_reviews:
    k =wt(each)
    elem=[]
    for each in k:
        if each not in stop_words and each.isalpha():
            elem.append(lm.lemmatize(each.lower()))
    wrdList.append(' '.join(elem))
#def max_features(i):
cv=CountVectorizer(max_features=300)
X=cv.fit_transform(wrdList).toarray()
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
params = dict(n_estimators=[10],criterion=['gini','entropy'],min_samples_split=[2,3,4,5,6],max_features=['auto',20,30,40,50],
             min_impurity_decrease=[0.0],max_leaf_nodes=[30,44,67,89],random_state=[0,13,43,49,47,37,38,18,9,27])
rf=RandomForestClassifier()
Y = dfNew.feedback
print(Y.shape,X.shape)
grid=GridSearchCV(estimator=rf,param_grid=params,cv=10,n_jobs=-1,scoring='accuracy')
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=3)
grid.fit(X_train,y_train)
print(grid.best_score_,grid.best_params_,grid.best_estimator_)

(3150,) (3150, 300)
0.930952380952381 {'criterion': 'entropy', 'max_features': 30, 'max_leaf_nodes': 89, 'min_impurity_decrease': 0.0, 'min_samples_split': 4, 'n_estimators': 10, 'random_state': 18} RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=30, max_leaf_nodes=89,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=18, verbose=0, warm_start=False)
